# Scoring model for LendingClub dataset - Technical notebook

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

#created methods for datacleaning
from loan_helper import data_cleaning
from loan_helper import data_converting

In [2]:
!ls -lath

total 152
drwxr-xr-x   3 Daniel  staff    96B Oct 21 13:23 __pycache__
drwxr-xr-x  11 Daniel  staff   352B Oct 21 13:23 .
drwxr-xr-x   2 Daniel  staff    64B Oct 21 13:21 .ipynb_checkpoints
drwxr-xr-x  14 Daniel  staff   448B Oct 21 13:21 .git
-rw-r--r--   1 Daniel  staff   2.1K Oct 21 13:21 loan_helper.py
-rw-r--r--   1 Daniel  staff    16K Oct 21 13:21 col_selection.xlsx
-rw-r--r--   1 Daniel  staff    44K Oct 21 13:21 Technical_Mod4.ipynb
-rw-r--r--   1 Daniel  staff   6.0K Oct 21 13:21 .DS_Store
-rw-r--r--   1 Daniel  staff    30B Oct 21 11:03 README.md
drwxr-xr-x   5 Daniel  staff   160B Oct 21 11:03 LendingClub
drwxr-xr-x  16 Daniel  staff   512B Oct 21 11:03 ..


#### LendingClub data source:

https://www.lendingclub.com/info/download-data.action

In [3]:
description = pd.read_excel('LendingClub/LCDataDictionary.xlsx')

In [6]:
# from zipfile import ZipFile
zip_file = ZipFile('LendingClub/LoanStats3d_securev1.csv.zip')
data_lc = pd.read_csv(zip_file.open('LoanStats3d_securev1.csv'), low_memory=False, header=1)

NameError: name 'ZipFile' is not defined

In [7]:
#read LendingClub loan data from 2015
data_lc = pd.read_csv('LendingClub/LoanStats3d_securev1.csv', low_memory=False, header=1)

FileNotFoundError: [Errno 2] File b'LendingClub/LoanStats3d_securev1.csv' does not exist: b'LendingClub/LoanStats3d_securev1.csv'

In [ ]:
#* * * IMPORTANT * * *
#removed two rows with full NAN values
data_lc = data_lc.loc[data_lc.loan_amnt.notnull()]
data_lc.shape

## Understanding the columns

In order to understand the columns a dataframe will be created ('desc') with column names, two examples, datatype, and the long description.

In [ ]:
col_selection = pd.read_excel('col_selection.xlsx')

In [ ]:
col_selection.head()

In [ ]:
selected_col = col_selection.loc[col_selection.Decision == 'keep', 'col_name'].to_list()
selected_col

## Feature selection and feature engineering

### First round feauture selection

Originally the dataset contained 150 columns. When we were reducing the features in order to avoid the overfitting the model we selected features according to the following:

- Discarded columns that contained payment or collection information (47 columns)
- Discarded columns that contained information that were not available at the time of credit application
- Discarded features that require too much data processing (typically free input i.e. emp_title)
- Discarded redundant features (subgrade - grade, title - purpose)
- Discarded feauters that contain too much NAN values (mnths_since_last_delinq, mths_since_recent_bc_dlq, mths_since_recent_revol_delinq)

In [ ]:
dataset = data_lc.loc[:, selected_col]

In [ ]:
dataset.shape

In [ ]:
dataset.info()

### Converting datatypes

- emp_length column to convert numeric and missing values replaced with average
- earliest credit line: convert date to numeric (years)
- revol_util (revolving utilization) convert to numeric

In [ ]:
dataset = data_converting(dataset)

In [ ]:
dataset.info()

## Deciding on the target feature, creating X and y

In [ ]:
dataset.loan_status.value_counts()

#### Meaning of the categories
<b>Fully paid:</b> Loan has been fully repaid, either at the expiration of the 3- or 5-year year term or as a result of a prepayment.

<b>Current:</b> Loan is up to date on all outstanding payments. 

<b>In Grace Period:</b> Loan is past due but within the 15-day grace period. 

<b>Late (16-30):</b> Loan has not been current for 16 to 30 days. Learn more about the tools LendingClub has to deal with delinquent borrowers.

<b>Late (31-120):</b> Loan has not been current for 31 to 120 days. Learn more about the tools LendingClub has to deal with delinquent borrowers.

<b>Default:</b> Loan has not been current for an extended period of time. Learn more about the difference between “default” and “charge off”.

<b>Charged Off:</b> Loan for which there is no longer a reasonable expectation of further payments. Upon Charge Off, the remaining principal balance of the Note is deducted from the account balance. Learn more about the difference between “default” and “charge off”.

Sosurce: https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

In [ ]:
pd.crosstab(columns=dataset['loan_status'], index=dataset['term'],)

We decided to ignore the 'gray' categories, where there might chance to the recovery of the loan. The 'Current' category contains the 60 months term loans, removing them would panalize the long term loans by increasing the default rate within this category. 

In [ ]:
#Select default categories:
dataset.loc[dataset.loan_status == 'Fully Paid', 'default'] = 0
dataset.loc[dataset.loan_status == 'Charged Off', 'default'] = 1
dataset.loc[dataset.loan_status == 'Current', 'default'] = 0

dataset = dataset.loc[dataset.default.notnull()]

In [ ]:
#remove loan_status, default replace it
dataset.drop(columns='loan_status', inplace=True)

In [ ]:
plt.figure(figsize=(6,5))
plt.bar(x=['non default', 'default'], height=dataset.default.value_counts()/len(dataset), width=0.6,)
plt.title('The distribution of defaulted and non defaulted loans\n')
ax=plt.gca();

In [ ]:
x_feats = dataset.columns.to_list()
x_feats.remove('default')

In [ ]:
len(x_feats)

In [ ]:
X = pd.get_dummies(dataset[x_feats], drop_first=True)

In [ ]:
y = dataset.default

In [ ]:
y.value_counts()

In [ ]:
X.head()

In [ ]:
len(X.columns)

## Train-Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0) #25%

## Further dataleaning separately for train and test

#### Summary:

- annual_inc: in case of annual income we had to handle extreme values (there are many strategies, we were choosing truncating the extreme values to the value of the 99.5 quantile)
